In [5]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit
from pprint import pprint
import pymongo

load_dotenv()

connector = DbConnector()

You are connected to the database: assignment3
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

### Task 2
Find the average, maximum and minimum number of trackpoints per user

##### Average

##### Maximum

##### Minimum

### Task 3

Find the top 15 users with the highest number of activities.

In [6]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 15},
    ]
)

for row in rows:
    pprint(row)

{'_id': '025', 'count': 715}
{'_id': '128', 'count': 519}
{'_id': '062', 'count': 406}
{'_id': '041', 'count': 399}
{'_id': '004', 'count': 346}
{'_id': '140', 'count': 345}
{'_id': '017', 'count': 265}
{'_id': '003', 'count': 261}
{'_id': '014', 'count': 236}
{'_id': '030', 'count': 210}
{'_id': '011', 'count': 201}
{'_id': '039', 'count': 198}
{'_id': '034', 'count': 180}
{'_id': '000', 'count': 155}
{'_id': '002', 'count': 146}


### Task 4
Find all users who have taken a bus.

### Task 5
List the top 10 users by their amount of different transportation modes.

### Task 6
Find activities that are registered multiple times. You should find the query even
if it gives zero result.

Assumption: Registered multiple times, meaning duplicates with the same user_id, transportation_mode, start_date_time, end_date_time.

In [7]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {
            "$lookup": {
                "from": "Activity",
                "let": {
                    "a_id": "$_id",
                    "a_user_id": "$user_id",
                    "a_transportation_mode": "$transportation_mode",
                    "a_start_date_time": "$start_date_time",
                    "a_end_date_time": "$end_date_time",
                },
                "pipeline": [
                    {
                        "$match": {
                            "$and": [
                                {"a_id": {
                                    "$ne":  "$_id"
                                }},
                                {"a_user_id": {
                                    "$eq": "$user_id"
                                }},
                                {
                                    "a_transportation_mode": {
                                        "$eq": "$transportation_mode"
                                    }
                                },
                                {
                                    "a_start_date_time": {
                                        "$eq":"$start_date_time"
                                    }

                                },
                                {
                                    "a_end_date_time": {
                                        "$eq":"$end_date_time"
                                    }

                                },
                            ]
                        }
                    }
                ],
                "as": "duplicates"
            }
        },
        {"$match": {
            "duplicates": {
                "$ne": []
            }
        }}
    ]
)

for row in rows:
    print(row)

### Task 7
a) Find the number of users that have started an activity in one day and ended the activity the next day.

b) List the transportation mode, user id and duration for these activities.

Assumption: As the question explicitly mentions transportation mode we do not count the rows in the Activity table with transportation mode equals NULL as relevant to this question. Thus, there are less rows in this sub task. However, if it is the case to include these NULL rows then we only need to remove the 'AND transportation_mode IS NOT NULL' part.

### Task 8

Find the number of users which have been close to each other in time and space. Close is defined as the same space (50 meters) and for the same half minute (30 seconds)

### Task 9
Find the top 15 users who have gained the most altitude meters.

In [23]:
collection = connector.db["User"]

rows = collection.aggregate(
    [
        {
            "$lookup": {
                "from": "Activity",
                "localField": "_id",
                "foreignField": "user_id",
                "as": "activity",
            }
        },
        {
            "$group": {
                "_id": {
                    "user_id": "$_id",
                    "transportation_mode": "activity.transportation_mode" 
                },
                "sum": {"$sum": 1},
            }
        },

    ]
)

for row in rows:
    pprint(row)

{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '177'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '023'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '052'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '051'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '083'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '110'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '135'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '099'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         'user_id': '128'},
 'sum': 1}
{'_id': {'transportation_mode': 'activity.transportation_mode',
         

In [17]:
collection = connector.db["TrackPoint"]
rows = collection.aggregate(
    [
        
        {
            "$setWindowFields": {
                "partitionBy": "$activity_id",
                "sortBy": {"datetime": 1},
                "output": {
                    "previous_altitude": {
                        "$shift": {"output": "$altitude", "by": -1}
                    }
                },
            }
        },
        {
            "$project": {
                "difference": {
                    "$subtract": ["$altitude", "$previous_altitude"]
                }
            }
        },
    ]
)

for row in rows:
    pprint(row)

OperationFailure: PlanExecutor error during aggregation :: caused by :: can't $subtract string from double, full error: {'ok': 0.0, 'errmsg': "PlanExecutor error during aggregation :: caused by :: can't $subtract string from double", 'code': 14, 'codeName': 'TypeMismatch'}

In [8]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {
            "$lookup": {
                "from": "TrackPoint",
                "localField": "_id",
                "foreignField": "activity_id",
                "as": "trackpoint",
            }
        },
        {"$match": {"trackpoint.altitude": {"$ne": -777}}},
        {
            "$setWindowFields": {
                "partitionBy": "trackpoint.activity_id",
                "sortBy": {"trackpoint.datetime": 1},
                "output": {
                    "previous_altitude": {
                        "$shift": {"output": "trackpoint.altitude", "by": -1}
                    }
                },
            }
        },
        {
            "$project": {
                "user_id"
                "difference": {
                    "$subtract": ["trackpoint.altitude", "$previous_altitude"]
                }
            }
        },
        {"$match": {"difference": {"$gt": 0}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                },
                "sum": {"$sum": "difference"},
            }
        },
        {
            "$project": {
                "user_id": 1,
                "altitude_in_meters": {"$multiply": ["sum", 0.304]},
            }
        },
        {"$sort": {"altitude_in_meters": -1}},
        {"$limit": 15},
    ]
)

for row in rows:
    pprint(row)

OperationFailure: PlanExecutor error during aggregation :: caused by :: can't $subtract string from array, full error: {'ok': 0.0, 'errmsg': "PlanExecutor error during aggregation :: caused by :: can't $subtract string from array", 'code': 14, 'codeName': 'TypeMismatch'}

### Task 10
Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 11
Find all users who have invalid activities, and the number of invalid activities per user.

### Task 12
Find all the users who have registered transportation_mode and their most used transportation_mode

In [ ]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode": {"$ne": None}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                    "transportation_mode": "$transportation_mode",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$lookup": {
                "from": "User",
                "let": {
                    "a_id": "$_id",
                    "a_has_labels": "$has_labels",
                },
                "pipeline": [{"$match": {"$expr": {"$eq": ["$$a_id", "$user_id"]}}}],
                "as": "saves_transportation",
            }
        },
        {"$match": {"saves_transportation": {"$ne": {"$a_has_labels": False}}}},
        {"$sort": {"_id.user_id": 1, "count": -1}},
        {
            "$group": {
                "_id": "$_id.user_id",
                "most_used_transportation_mode": {"$first": "$_id.transportation_mode"},
            }
        },
        {"$sort": {"_id.user_id": 1}},
    ]
)

for row in rows:
    pprint(row)

{'_id': '117', 'most_used_transportation_mode': 'walk'}
{'_id': '020', 'most_used_transportation_mode': 'bike'}
{'_id': '065', 'most_used_transportation_mode': 'bike'}
{'_id': '097', 'most_used_transportation_mode': 'bike'}
{'_id': '062', 'most_used_transportation_mode': 'bus'}
{'_id': '107', 'most_used_transportation_mode': 'walk'}
{'_id': '161', 'most_used_transportation_mode': 'walk'}
{'_id': '167', 'most_used_transportation_mode': 'bike'}
{'_id': '021', 'most_used_transportation_mode': 'walk'}
{'_id': '125', 'most_used_transportation_mode': 'bike'}
{'_id': '058', 'most_used_transportation_mode': 'walk'}
{'_id': '128', 'most_used_transportation_mode': 'car'}
{'_id': '139', 'most_used_transportation_mode': 'bike'}
{'_id': '089', 'most_used_transportation_mode': 'car'}
{'_id': '052', 'most_used_transportation_mode': 'bus'}
{'_id': '092', 'most_used_transportation_mode': 'walk'}
{'_id': '115', 'most_used_transportation_mode': 'car'}
{'_id': '163', 'most_used_transportation_mode': 'bike